In [1]:
# imports

import os
import sys
sys.path.append('/mnt/0A2AAC152AABFBB7/CGE/luxgiant-clinical')
import luxgiant_clinical.TwoCatAnalysis as two

import pandas as pd
import numpy as np

In [2]:
# load data
folder_path = '/mnt/0A2AAC152AABFBB7/CGE/luxgiant-clinical/data'

df = pd.read_csv(
    os.path.join(folder_path, 'cleaned_file.csv'), 
    low_memory=False
)

In [3]:
# filter data to keep only patients with gender information
mask_patients= (df['Status']=='Patient')
mask_gender  = (~df['sex'].isnull()) 

df_cases = df[mask_patients & mask_gender].reset_index(drop=True)
del df

df_cases.shape

(7918, 716)

In [4]:

df_on = df_cases[df_cases['hyonoff']=='On'].reset_index(drop=True)

df_on['Stage I-III']= df_on['hystage'].map({'Not severe':1, 'Severe':0})
df_on['Stage IV-V'] = df_on['hystage'].map({'Not severe':0, 'Severe':1})
df_on['UPDRS III-ON']= df_on['updrs_part_iii_total_score'].copy()

variables = ['participant_id', 'sex', 'Stage I-III', 'Stage IV-V', 'UPDRS III-ON']

stats_meas1 = {
    'n': ['Stage I-III', 'Stage IV-V'],
    'median' : ['UPDRS III-ON']
}
groups = ['Male', 'Female']

# keep only variables for analysis

df_on = df_on[variables].copy()

In [5]:
df_off = df_cases[df_cases['hyonoff']=='Off'].reset_index(drop=True)

df_off['Stage I-III']= df_off['hystage'].map({'Not severe':1, 'Severe':0})
df_off['Stage IV-V'] = df_off['hystage'].map({'Not severe':0, 'Severe':1})
df_off['UPDRS III-OFF']= df_off['updrs_part_iii_total_score'].copy()

variables = ['participant_id', 'sex', 'Stage I-III', 'Stage IV-V', 'UPDRS III-OFF']

stats_meas2 = {
    'n': ['Stage I-III', 'Stage IV-V'],
    'median' : ['UPDRS III-OFF']
}

# keep only variables for analysis

df_off = df_off[variables].copy()

In [6]:
summary1 = [
    two.report_proportion(
        data_df    =df_off,
        variables  =stats_meas2['n'],
        groups     =groups,
        grouping_by='sex',
        subheader  ='H & Y OFF'
        ),
    two.report_proportion(
        data_df    =df_on,
        variables  =stats_meas1['n'],
        groups     =groups,
        grouping_by='sex',
        subheader  ='H & Y ON'
    ),
    two.report_median_iqr(
        data_df    =df_on,
        variables  =stats_meas1['median'],
        groups     =groups,
        grouping_by='sex'
    ),
    two.report_median_iqr(
        data_df    =df_off,
        variables  =stats_meas2['median'],
        groups     =groups,
        grouping_by='sex'
    )
]

In [7]:
variables_dict = {
    "total_score_for_moca": "MOCA Score",
    "total_score_for_bdi" : "BDI Score" 
}
variables_1 = ['participant_id', 'sex'] + list(variables_dict.keys())

# statistical measures
stats_meas3 = {
    'mean'  : ["MOCA Score"],
    'median': ["BDI Score"]
}

groups = ['Male', 'Female']

# keep only variables for analysis

df_cases_1 = df_cases[variables_1].copy()
df_cases_1 = df_cases_1.rename(columns=variables_dict)

In [8]:
summary2 = [ 
    two.report_mean_std(
        data_df    =df_cases_1,
        variables  =stats_meas3['mean'],
        groups     =groups,
        grouping_by='sex'
    ),
    two.report_median_iqr(
        data_df    =df_cases_1,
        variables  =stats_meas3['median'],
        groups     =groups,
        grouping_by='sex'
    )
]

In [9]:
df_3 = pd.concat(
    summary1 + summary2, axis=0, ignore_index=True
)
df_3 = two.final_formatter(df_3, groups)

df_3.to_csv(os.path.join(folder_path, 'Table_3.csv'), index=False)
df_3

,Variable,Statistical Measure,Male,Female,p-value,Total,Available Samples for Analysis
0,H & Y OFF,,,,,,
1,Stage I-III,n (%),1642 (94.4),795 (93.4),0.3377,2437 (94.1),2591
2,Stage IV-V,n (%),98 (5.6),56 (6.6),0.3377,154 (5.9),2591
3,H & Y ON,,,,,,
4,Stage I-III,n (%),2537 (95.8),1200 (95.4),0.584,3737 (95.6),3907
5,Stage IV-V,n (%),112 (4.2),58 (4.6),0.584,170 (4.4),3907
6,UPDRS III-ON,median (IQR),21.0 (13.0 - 30.0),20.0 (13.0 - 29.0),0.0582,21.0 (13.0 - 30.0),3223
7,UPDRS III-OFF,median (IQR),38.0 (27.0 - 46.0),37.0 (26.0 - 46.0),0.3854,37.0 (27.0 - 46.0),1562
8,MOCA Score,mean (SD),24.1 (5.2),22.8 (6.2),p<0.001,23.7 (5.5),5301
9,BDI Score,median (IQR),8.0 (5.0 - 13.0),9.0 (6.0 - 14.0),p<0.001,9.0 (5.0 - 13.0),5850
